In [1]:
#IMPORTS
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import preprocessing

## Part 3: VALIDATOR to test
modifier and simulator working in dynamic way, with hardcoded ranges
* next step, extract the ranges 



In [2]:
## VALIDATOR FILE: :
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import StandardScaler
from global_settings import fitting_threshold, mdv

class Validator:
    def __init__(self):
        self.iterations = 1
        self.total_points = 0
        self.total_bad_points = 0
        self.range = (mdv["domain_min_range"], mdv["domain_max_range"])
        self.num_points_evaluated = 0
        self.history = []  # Initialize history as an empty list

    @staticmethod
    def fit_curve(x_values, y_values, global_range=0):
        print('       *** USING fit_curve')
        
        x_values = np.array(x_values)  # Convert to numpy array
        y_values = np.array(y_values)  # Convert to numpy array

        # fit model
        model = HuberRegressor(epsilon=1)
        model.fit(x_values.reshape(-1, 1), y_values)

        # Get the coefficients of the fitted line
        slope = model.coef_[0]
        intercept = model.intercept_
        print('       *** OUTPUT fit_curve slope, intercept',slope, intercept,'\n')

        return slope, intercept
    
    @staticmethod
    def generate_ranges_from_unfit_points(unfit_points, threshold):
        print('       *** USING generate_ranges_from_unfit_points')
        unfit_points.sort()  # Sort the unfit points based on x-values
        unfit_ranges = []

        current_start, current_end = unfit_points[0][0] - threshold, unfit_points[0][0] + threshold

        for i in range(1, len(unfit_points)):
            if unfit_points[i][0] - current_end <= threshold:
                current_end = unfit_points[i][0] + threshold
            else:
                unfit_ranges.append((current_start, current_end))
                current_start, current_end = unfit_points[i][0] - threshold, unfit_points[i][0] + threshold

        unfit_ranges.append((current_start, current_end))

        # Combine overlapping ranges
        combined_ranges = []

        for start, end in unfit_ranges:
            if combined_ranges and start <= combined_ranges[-1][1] + threshold:
                combined_ranges[-1] = (combined_ranges[-1][0], max(combined_ranges[-1][1], end))
            else:
                combined_ranges.append((start, end))

        print('       *** OUTPUT unfit_ranges', combined_ranges, '\n')
        return combined_ranges
    
    
    @staticmethod
    def get_unfitting_point(x_values, y_values, fitted_curve, threshold=0.9):
        print('       *** USING get_unfitting_point')
        temp_soln = [1, 4, 5, 6]
        unfit_points = [(x_values[i], y_values[i]) for i in temp_soln]
        print('       *** OUTPUT unfit_points',unfit_points,'\n')
        return unfit_points


    def update_num_points_evaluated(self, points, min_x, max_x):
        self.num_points_evaluated = len([(x, y) for x, y in points if min_x <= x <= max_x])

    @staticmethod
    def local_exploration_validator_A(x_values, y_values, global_range=[mdv["domain_min_range"], mdv["domain_max_range"]], threshold=fitting_threshold):
        print('       *** USING local_exploration_validator_A')
        fitted_curve = Validator.fit_curve(x_values, y_values, global_range)
        unfit_points = Validator.get_unfitting_point(x_values, y_values, fitted_curve, threshold=threshold)
        unfitting_ranges = Validator.generate_ranges_from_unfit_points(unfit_points, threshold=threshold)
        # print(unfitting_ranges)
        # Validator.update_num_points_evaluated(len(x_values), min=global_range[0], max=global_range[1])
        print('       *** OUTPUT unfitting_ranges',unfitting_ranges,'\n')
        return unfitting_ranges

    @staticmethod
    def validator_controller(mod_x_list, sim_y_list, global_range=[mdv["domain_min_range"], mdv["domain_max_range"]], threshold=fitting_threshold, local_validator=local_exploration_validator_A, do_plot=False):
        print('       *** USING validator_controller')

        validator_ranges = local_validator(mod_x_list, sim_y_list, global_range=[mdv["domain_min_range"], mdv["domain_max_range"]], threshold=threshold)
        print('       *** OUTPUT validator_ranges',validator_ranges,'\n')
        return validator_ranges

    @staticmethod
    def thresholding(fitted_curve, threshold):
        print('       *** USING thresholding')
        x_values = np.arange(len(fitted_curve))
        y_values = np.array(fitted_curve)
        above_threshold = x_values[y_values > threshold]
        print('       *** OUTPUT above_threshold',above_threshold,'\n')
        return above_threshold

# For the purpose of testing just the validator, let's use this list: 
import numpy as np
from global_settings import mdv
# from Validator import Validator
from Modifier import Modifier
from Simulator import Simulator

# Generate x values
mod_x = np.linspace(1,20,20)
# print(mod_x)
# Generate y values resembling a parabola shape
sim_y_list = mod_x*2

# Add four outliers
outliers_indices = [6, 8, 11, 14]
outliers_values = np.array([0.8, 1.5, 1.2, 1.6])

sim_y_list[outliers_indices] = outliers_values
print('** Sim y and Mod x output\n', sim_y_list,'\n',mod_x,'\n') 

output = Validator.validator_controller(mod_x_list=mod_x,sim_y_list=sim_y_list,threshold=0.9)
print('** Validator output\n', output,'\n\n') 



** Sim y and Mod x output
 [ 2.   4.   6.   8.  10.  12.   0.8 16.   1.5 20.  22.   1.2 26.  28.
  1.6 32.  34.  36.  38.  40. ] 
 [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20.] 

       *** USING validator_controller
       *** USING local_exploration_validator_A
       *** USING fit_curve
       *** OUTPUT fit_curve slope, intercept 1.9999999961608057 2.601417701873449e-08 

       *** USING get_unfitting_point
       *** OUTPUT unfit_points [(2.0, 4.0), (5.0, 10.0), (6.0, 12.0), (7.0, 0.8)] 

       *** USING generate_ranges_from_unfit_points
       *** OUTPUT unfit_ranges [(1.1, 2.9), (4.1, 7.9)] 

       *** OUTPUT unfitting_ranges [(1.1, 2.9), (4.1, 7.9)] 

       *** OUTPUT validator_ranges [(1.1, 2.9), (4.1, 7.9)] 

** Validator output
 [(1.1, 2.9), (4.1, 7.9)] 




In [10]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np

class ValidatorGUI:
    def __init__(self):
        self.mod_x = None
        self.sim_y = None
        self.threshold = None

        self.mod_x_input = widgets.FloatRangeSlider(
            value=[1, 20],
            min=1,
            max=20,
            step=1,
            description='Mod X Range:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )

        self.sim_y_input = widgets.Text(
            value='1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20',
            description='Sim Y Values:',
            disabled=False
        )

        self.threshold_input = widgets.FloatSlider(
            value=0.9,
            min=0,
            max=1.0,
            step=0.01,
            description='Threshold:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
        )

        self.run_button = widgets.Button(description="Run Validator")
        self.generate_random_button = widgets.Button(description="Generate Random Data")

        self.run_button.on_click(self.run_validator)
        self.generate_random_button.on_click(self.generate_random_data)

        self.display_widgets()

    def display_widgets(self):
        display(self.mod_x_input, self.sim_y_input, self.threshold_input, self.run_button, self.generate_random_button)

    def run_validator(self, b):
        self.mod_x = np.linspace(*self.mod_x_input.value, num=20)
        self.sim_y = np.array(list(map(float, self.sim_y_input.value.split())))
        self.threshold = self.threshold_input.value

        output = Validator.validator_controller(mod_x_list=self.mod_x, sim_y_list=self.sim_y, threshold=self.threshold)
        print('** Validator output\n', output, '\n\n')

    def generate_random_data(self, b):
        self.mod_x = np.linspace(1, 20, 20)
        self.sim_y = np.random.randint(1, 21, size=20)
        self.mod_x_input.value = [1, 20]
        self.sim_y_input.value = ' '.join(map(str, self.sim_y))
        self.threshold_input.value = 0.9
        print('Generated random data.')

gui = ValidatorGUI()


FloatRangeSlider(value=(1.0, 20.0), continuous_update=False, description='Mod X Range:', max=20.0, min=1.0, re…

Text(value='1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20', description='Sim Y Values:')

FloatSlider(value=0.9, continuous_update=False, description='Threshold:', max=1.0, step=0.01)

Button(description='Run Validator', style=ButtonStyle())

Button(description='Generate Random Data', style=ButtonStyle())

       *** USING validator_controller
       *** USING local_exploration_validator_A
       *** USING fit_curve
       *** OUTPUT fit_curve slope, intercept 1.0000000000142146 -1.4786564256038742e-10 

       *** USING get_unfitting_point
       *** OUTPUT unfit_points [(2.0, 2.0), (5.0, 5.0), (6.0, 6.0), (7.0, 7.0)] 

       *** USING generate_ranges_from_unfit_points
       *** OUTPUT unfit_ranges [(1.1, 2.9), (4.1, 7.9)] 

       *** OUTPUT unfitting_ranges [(1.1, 2.9), (4.1, 7.9)] 

       *** OUTPUT validator_ranges [(1.1, 2.9), (4.1, 7.9)] 

** Validator output
 [(1.1, 2.9), (4.1, 7.9)] 


Generated random data.
       *** USING validator_controller
       *** USING local_exploration_validator_A
       *** USING fit_curve


ValueError: HuberRegressor convergence failed: l-BFGS-b solver terminated with ABNORMAL_TERMINATION_IN_LNSRCH

Generated random data.
       *** USING validator_controller
       *** USING local_exploration_validator_A
       *** USING fit_curve
       *** OUTPUT fit_curve slope, intercept 0.1666678873704096 7.499989013634126 

       *** USING get_unfitting_point
       *** OUTPUT unfit_points [(2.0, 10.0), (5.0, 10.0), (6.0, 16.0), (7.0, 15.0)] 

       *** USING generate_ranges_from_unfit_points
       *** OUTPUT unfit_ranges [(1.1, 2.9), (4.1, 7.9)] 

       *** OUTPUT unfitting_ranges [(1.1, 2.9), (4.1, 7.9)] 

       *** OUTPUT validator_ranges [(1.1, 2.9), (4.1, 7.9)] 

** Validator output
 [(1.1, 2.9), (4.1, 7.9)] 


Generated random data.
       *** USING validator_controller
       *** USING local_exploration_validator_A
       *** USING fit_curve
       *** OUTPUT fit_curve slope, intercept 0.12586593393233378 11.860279127915389 

       *** USING get_unfitting_point
       *** OUTPUT unfit_points [(2.0, 12.0), (5.0, 20.0), (6.0, 17.0), (7.0, 8.0)] 

       *** USING generate_ranges_

## PART 4: Main Func: with Modifier.py

In [8]:
# PLACEHOLDER FOR THE MAIN FUNCTION V2
%load_ext autoreload
%autoreload 2

from global_settings import mdv
# from Validator import Validator
from Modifier import Modifier
from Simulator import Simulator

print('* MDV: \n',mdv,'\n')
#TODO: How to autoreload without restarting kernel (for global vars)
validate = Validator
modifier = Modifier
simulator = Simulator
appendedvars=[]

mainfunc=True
# Initialize interval list
interval_lists=[(mdv["domain_min_range"], mdv["domain_max_range"])]
x=1
while mainfunc==True:
    mod_x_list= modifier.modifier_controller(range_list=interval_lists,local_modifier=modifier.local_modifier_A, do_plot=False)

    if mod_x_list == False: # FALSE IF ["modifier_data_point"] < mdv["modifier_incremental_unit"]:
        print('*   ITERATIONS END HERE   *')
        break

    mod_x,sim_y_list = simulator.simulator_controller(mod_x_list,selected_function=simulator.sim_func_A)
    assert len(mod_x) == len(sim_y_list)
    
    
    print('\nValidator ...')
    ranges = Validator.validator_controller(mod_x_list=mod_x,sim_y_list=sim_y_list,threshold=0.9)
    print('** Validator output\n', ranges,'\n\n') 
 



* MDV: 
 {'domain_min_range': 1, 'domain_max_range': 100, 'modifier_incremental_unit': 3, 'modifier_data_point': 10} 


Modifier controller...
  * Interval:  [(1, 100)]
     * Iterations within Modifier:  0
  * Mod_x shape:    (1, 10)

Simulator...
  * Sim_y shape:    (10,)

Validator ...
       *** USING validator_controller
       *** USING local_exploration_validator_A
       *** USING fit_curve
       *** OUTPUT fit_curve slope, intercept 7128.86031868948 -94476.63902855631 

       *** USING get_unfitting_point
       *** OUTPUT unfit_points [(2.4347826086956523, -5909.131153994845), (21.08695652173913, -18856.749454793862), (32.086956521739125, 28613.96194414556), (45.47826086956522, 78764.5243781414)] 

       *** USING generate_ranges_from_unfit_points
       *** OUTPUT unfit_ranges [(1.5347826086956524, 3.3347826086956522), (20.18695652173913, 21.986956521739128), (31.186956521739127, 32.986956521739124), (44.57826086956522, 46.37826086956522)] 

       *** OUTPUT unfitting_ra